In [58]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('json')

from sublimpy import turbulence
import matplotlib.pyplot as plt
from sublimpy import tidy
import metpy.constants

# Open data

## Open SOS Measurement Dataset

In [59]:
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df_fn = f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet'

tidy_df = pd.read_parquet(tidy_df_fn)

# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:end_date].reset_index()

# Open z0 datasets - created with `calculate_roughness_length.ipynb`

In [60]:
z0_andreas_df = pd.read_parquet("z0estimates/z0_andreas_df.parquet")
z0_andreas_df_weekly = pd.read_parquet("z0estimates/z0_andreas_df_weekly.parquet")

In [61]:
time_series = tidy_df.query("variable == 'u*_3m_c'").time

# reindex the weekly data so it fills the 30 minute timesteps
z0_andreas_df_weekly = z0_andreas_df_weekly.set_index('time').reindex(time_series, method ='bfill')

# backfill the 30-minute data, so we can still use z0 values when we had to remove them
z0_andreas_df['z0'] = z0_andreas_df['z0'].interpolate('bfill')
z0_andreas_df['z0T'] = z0_andreas_df['z0'].interpolate('bfill')
z0_andreas_df['z0q'] = z0_andreas_df['z0'].interpolate('bfill')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_59459/4237257618.py:7: FutureWarning: Series.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  z0_andreas_df['z0'] = z0_andreas_df['z0'].interpolate('bfill')
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_59459/4237257618.py:8: FutureWarning: Series.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  z0_andreas_df['z0T'] = z0_andreas_df['z0'].interpolate('bfill')
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_59459/4237257618.py:9: FutureWarning: Series.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  z0_andreas_df['z0q'] = z0_andreas_df['z0'].interpolate('bfill')


In [45]:
tidy_df = tidy.tidy_df_add_variable(
    tidy_df,
    z0_andreas_df.z0,
    variable='z0_andreas',
    measurement='z0',
    height=3,
    tower='c'
)

tidy_df = tidy.tidy_df_add_variable(
    tidy_df,
    z0_andreas_df_weekly['all data'],
    variable='z0_andreas_weekly',
    measurement='z0',
    height=3,
    tower='c'
)

In [46]:
src = tidy_df.query("measurement == 'z0'")
alt.Chart(src).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q").scale(type='log', domain=[1e-8, 1e-1], clamp=True),
    alt.Color('variable:O')
).properties(width=1000)

alt.Chart(...)

In [47]:
tidy_df.to_parquet(tidy_df_fn, index=False)

In [48]:
ls -lah | grep tidy | grep parquet

-rw-r--r--@   1 elischwat  staff    33M Mar  6 12:24 tidy_df_20221130_20230509_noplanar_fit.parquet
-rw-r--r--    1 elischwat  staff   257M Feb 20 22:02 tidy_df_20221130_20230509_noplanar_fit_ARCHIVE.parquet
-rw-r--r--@   1 elischwat  staff    45M Mar  6 15:45 tidy_df_20221130_20230509_noplanar_fit_clean.parquet
